<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: Юрий Исаков и Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Тема 4. Линейные модели классификации и регрессии
## <center>  Практика. Идентификация пользователя с помощью логистической регрессии

Тут мы воспроизведем парочку бенчмарков нашего соревнования и вдохновимся побить третий бенчмарк, а также остальных участников. Веб-формы для отправки ответов тут не будет, ориентир – [leaderboard](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard) соревнования.

In [1]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

### 1. Загрузка и преобразование данных
Зарегистрируйтесь на [Kaggle](www.kaggle.com), если вы не сделали этого раньше, зайдите на [страницу](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования и скачайте данные. Первым делом загрузим обучающую и тестовую выборки и посмотрим на данные.

In [2]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv('../../data/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/train_sessions.csv',
                       index_col='session_id',
                       parse_dates=True)
test_df = pd.read_csv('../../data/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/test_sessions.csv',
                      index_col='session_id',
                      parse_dates=True)

# приведем колонки time1, ..., time10 к временному формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by='time1')

# посмотрим на заголовок обучающей выборки
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


In [3]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82797 entries, 1 to 82797
Data columns (total 20 columns):
site1     82797 non-null int64
time1     82797 non-null datetime64[ns]
site2     81308 non-null float64
time2     81308 non-null datetime64[ns]
site3     80075 non-null float64
time3     80075 non-null datetime64[ns]
site4     79182 non-null float64
time4     79182 non-null datetime64[ns]
site5     78341 non-null float64
time5     78341 non-null datetime64[ns]
site6     77566 non-null float64
time6     77566 non-null datetime64[ns]
site7     76840 non-null float64
time7     76840 non-null datetime64[ns]
site8     76151 non-null float64
time8     76151 non-null datetime64[ns]
site9     75484 non-null float64
time9     75484 non-null datetime64[ns]
site10    74806 non-null float64
time10    74806 non-null datetime64[ns]
dtypes: datetime64[ns](10), float64(9), int64(1)
memory usage: 13.3 MB


In [4]:
test_df.shape

(82797, 20)

В обучающей выборке содержатся следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - target – целевая переменная, 1 для сессий Элис, 0 для сессий других пользователей
    
Сессии пользователей выделены таким образом, что они не могут быть длиннее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд либо когда сессия заняла по времени более 30 минут.

В таблице встречаются пропущенные значения, это значит, что сессия состоит менее, чем из 10 сайтов. Заменим пропущенные значения нулями и приведем признаки к целому типу. Также загрузим словарь сайтов и посмотрим, как он выглядит:

In [5]:
# приведем колонки site1, ..., site10 к целочисленному формату и заменим пропуски нулями
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# загрузим словарик сайтов
with open(r"../../data/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(list(site_dict.keys()), 
                          index=list(site_dict.values()), 
                          columns=['site'])
print(u'всего сайтов:', sites_dict_df.shape[0])
sites_dict_df.head()

всего сайтов: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


Выделим целевую переменную и объединим выборки, чтобы вместе привести их к разреженному формату.

In [6]:
# наша целевая переменная
y_train = train_df['target']

# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]

Для самой первой модели будем использовать только посещенные сайты в сессии (но не будем обращать внимание на временные признаки). За таким выбором данных для модели стоит такая идея:  *у Элис есть свои излюбленные сайты, и чем чаще вы видим эти сайты в сессии, тем выше вероятность, что это сессия Элис и наоборот.*

Подготовим данные, из всей таблицы выберем только признаки `site1, site2, ... , site10`. Напомним, что пропущенные значения заменены нулем. Вот как выглядят первые строки таблицы:

In [7]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


Сессии представляют собой последовательность индексов сайтов и данные в таком виде неудобны для линейных методов. В соответствии с нашей гипотезой (у Элис есть излюбленные сайты) надо преобразовать эту таблицу таким образом, чтобы каждому возможному сайту соответствовал свой отдельный признак (колонка), а его значение равнялось бы количеству посещений этого сайта в сессии. Это делается в две строчки:

In [8]:
from scipy.sparse import csr_matrix

In [9]:
# csr_matrix?

In [10]:
# последовательность с индексами
sites_flatten = full_sites.values.flatten()
# print(full_sites.values.flatten())

# искомая матрица
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0] + 10, 10)))[:, 1:]
# print(full_sites_sparse)

In [11]:
X_train_sparse = full_sites_sparse[:idx_split]
X_test_sparse = full_sites_sparse[idx_split:]

In [12]:
X_train_sparse.shape, y_train.shape

((253561, 48371), (253561,))

In [13]:
X_test_sparse.shape

(82797, 48371)

Еще один плюс использования разреженных матриц в том, что для них имеются специальные реализации как матричных операций, так и алгоритмов машинного обучения, что подчас позволяет ощутимо ускорить операции за счет особенностей структуры данных. Это касается и логистической регрессии. Вот теперь у нас все готово для построения нашей первой модели.

### 2. Построение первой модели

Итак, у нас есть алгоритм и данные для него, построим нашу первую модель, воспользовавшись релизацией [логистической регрессии](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) из пакета `sklearn` с параметрами по умолчанию. Первые 90% данных будем использовать для обучения (обучающая выборка отсортирована по времени), а оставшиеся 10% для проверки качества (validation). 

**Напишите простую функцию, которая будет возвращать качество модели на отложенной выборке, и обучите наш первый классификатор**.

In [14]:
def get_auc_lr_valid(X, y, C=1.0, ratio = 0.9, seed=17):
    '''
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state 
              логистической регрессии
    '''
    
    x_train_len = int(ratio * X.shape[0])
    X_train = X[: x_train_len, :]
    X_test = X[x_train_len :, :]
    
    Y_train = y[: x_train_len]
    Y_test = y[x_train_len :]
    
    logit = LogisticRegression(n_jobs=-1, C=C, random_state=seed, solver='lbfgs')
    
    logit.fit(X_train, Y_train)
    
    pred_valid = logit.predict_proba(X_test)[:, 1]
    return roc_auc_score(Y_test, pred_valid)

**Заведём функцию для проверки числа правильных ответов (как в уроке по Decision tree)**

In [15]:
def get_accuracy_value(X, y, C=1.0, ratio = 0.7, seed=17):
    '''
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state 
              логистической регрессии
    '''
    
    x_train_len = int(ratio * X.shape[0])
    X_train = X[: x_train_len, :]
    X_test = X[x_train_len :, :]
    
    Y_train = y[: x_train_len]
    Y_test = y[x_train_len :]
    
    logit = LogisticRegression(n_jobs=-1, C=C, random_state=seed, solver='lbfgs')
    
    logit.fit(X_train, Y_train)
    
    pred = logit.predict(X_test)
    
    return accuracy_score(Y_test, pred)

**Посмотрите, какой получился ROC AUC на отложенной выборке.**

In [16]:
get_auc_lr_valid(X=X_train_sparse, y=y_train)

0.9197955574958127

Будем считать эту модель нашей первой отправной точкой (baseline). Для построения модели для прогноза на тестовой выборке **необходимо обучить модель заново уже на всей обучающей выборке** (пока наша модель обучалась лишь на части данных), что повысит ее обобщающую способность:

In [17]:
# функция для записи прогнозов в файл
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

**Обучите модель на всей выборке, сделайте прогноз для тестовой выборки и сделайте посылку в соревновании**.

In [18]:
logit = LogisticRegression(n_jobs=-1, C=1, random_state=17, solver='lbfgs')
logit.fit(X_train_sparse, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=-1, penalty='l2', random_state=17,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [19]:
pred_probabilities = logit.predict_proba(X_test_sparse)
pred_probabilities[:, 1]

array([2.21954270e-03, 2.51892726e-09, 6.16010896e-09, ...,
       8.43147766e-03, 3.87735238e-04, 1.29530564e-05])

In [20]:
# series = pd.Series(data=pred_probabilities[:, 1], index=range(1, pred_probabilities.shape[0] + 1))
# series.to_csv(path_or_buf="../../data/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/benchmark_1.csv",
#               header=["target"],
#               index_label="session_id")

Если вы выполните эти действия и загрузите ответ на [странице](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования, то воспроизведете первый бенчмарк "Logit".

### 3. Улучшение модели, построение новых признаков

Создайте такой признак, который будет представлять собой число вида ГГГГММ от той даты, когда проходила сессия, например 201407 -- 2014 год и 7 месяц. Таким образом, мы будем учитывать помесячный [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь период предоставленных данных.

In [21]:
new_train_features = pd.DataFrame(index=train_df.index)
new_test_features = pd.DataFrame(index=test_df.index)

In [22]:
new_train_features.shape, new_test_features.shape

((253561, 0), (82797, 0))

In [23]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 21669 to 204762
Data columns (total 21 columns):
site1     253561 non-null int64
time1     253561 non-null datetime64[ns]
site2     253561 non-null int64
time2     250098 non-null datetime64[ns]
site3     253561 non-null int64
time3     246919 non-null datetime64[ns]
site4     253561 non-null int64
time4     244321 non-null datetime64[ns]
site5     253561 non-null int64
time5     241829 non-null datetime64[ns]
site6     253561 non-null int64
time6     239495 non-null datetime64[ns]
site7     253561 non-null int64
time7     237297 non-null datetime64[ns]
site8     253561 non-null int64
time8     235224 non-null datetime64[ns]
site9     253561 non-null int64
time9     233084 non-null datetime64[ns]
site10    253561 non-null int64
time10    231052 non-null datetime64[ns]
target    253561 non-null int64
dtypes: datetime64[ns](10), int64(11)
memory usage: 42.6 MB


In [24]:
time = ['time%d' % d for d in range(1, 11)]
train_na_zero = train_df[time].fillna(value=pd.Timestamp(0), inplace=False)
test_na_zero = test_df[time].fillna(value=pd.Timestamp(0), inplace=False)

In [25]:
train_na_zero.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 21669 to 204762
Data columns (total 10 columns):
time1     253561 non-null datetime64[ns]
time2     253561 non-null datetime64[ns]
time3     253561 non-null datetime64[ns]
time4     253561 non-null datetime64[ns]
time5     253561 non-null datetime64[ns]
time6     253561 non-null datetime64[ns]
time7     253561 non-null datetime64[ns]
time8     253561 non-null datetime64[ns]
time9     253561 non-null datetime64[ns]
time10    253561 non-null datetime64[ns]
dtypes: datetime64[ns](10)
memory usage: 21.3 MB


In [26]:
# Рассмотрим только время начала каждой сессии - поле time1
new_train_features['year_month'] = train_na_zero['time1'].apply(lambda ts: str(ts.year * 100 + ts.month))
new_test_features['year_month'] = test_na_zero['time1'].apply(lambda ts: str(ts.year * 100 + ts.month))

new_train_features.head()

,year_month
session_id,
21669,201301
54843,201301
77292,201301
114021,201301
146670,201301


In [27]:
scaler = StandardScaler()
scaler.fit(new_train_features['year_month'].values.reshape(-1, 1))

new_train_features['year_month_scaled'] = scaler.transform(new_train_features['year_month'].values.reshape(-1, 1))
new_test_features['year_month_scaled'] = scaler.transform(new_test_features['year_month'].values.reshape(-1, 1))

In [28]:
new_train_features.head()

,year_month,year_month_scaled
session_id,,
21669,201301,-1.744405
54843,201301,-1.744405
77292,201301,-1.744405
114021,201301,-1.744405
146670,201301,-1.744405


Добавьте новый признак, предварительно отмасштабировав его с помощью `StandardScaler`, и снова посчитайте ROC AUC на отложенной выборке.

In [29]:
# Нужно объединить новые данные с существующими в разреженной матрице X_train_sparse
# Делаем это с помощью функции hstack из библиотеки scipy.sparse
# Требуется дополнительное явное преобразование в csr_matrix
X_train_sparse_new = csr_matrix(hstack([X_train_sparse,
                                new_train_features['year_month_scaled'].values.reshape(-1, 1)]))
X_train_sparse_new.shape # (253561, 48372)

# Аналогично для тестового набора
X_test_sparse_new = hstack([X_test_sparse,
                             new_test_features['year_month_scaled'].values.reshape(-1, 1)])
X_test_sparse_new.shape # (82797, 48372)
X_train_sparse.shape, X_train_sparse_new.shape # Добавился дополнительный признак в shape[1]

((253561, 48371), (253561, 48372))

In [30]:
%%time
get_auc_lr_valid(X=X_train_sparse_new, y=y_train)

CPU times: user 101 ms, sys: 117 ms, total: 218 ms
Wall time: 3.4 s


0.9198902054055882

**Добавьте два новых признака: start_hour и morning.**

Признак `start_hour` – это час в который началась сессия (от 0 до 23), а бинарный признак `morning` равен 1, если сессия началась утром и 0, если сессия началась позже (будем считать, что утро это если `start_hour равен` 11 или меньше).

**Посчитйте ROC AUC на отложенной выборке для выборки с:**
- сайтами, `start_month` и `start_hour`
- сайтами, `start_month` и `morning`
- сайтами, `start_month`, `start_hour` и `morning`

In [31]:
# start_month + start_hour
# Создадим похожую фичу для идентификации, в какое время суток предполагаемый человек начинает сессию
new_train_features['start_hour'] = train_na_zero['time1'].apply(lambda ts: str(ts.hour))
new_test_features['start_hour'] = test_na_zero['time1'].apply(lambda ts: str(ts.hour))

# Фича с учётом года и месяца работает хуже
# new_train_features['start_hour'] = train_na_zero['time1'].apply(lambda ts: str((ts.year * 100 + ts.month) * 100 + ts.hour))
# new_test_features['start_hour'] = test_na_zero['time1'].apply(lambda ts: str((ts.year * 100 + ts.month) * 100 + ts.hour))

# start_month + start_hour + morning
# Аналогичную операцию проделаем для признака morning
new_train_features['morning'] = train_na_zero['time1'].apply(lambda ts: int(ts.hour <= 11))
new_test_features['morning'] = test_na_zero['time1'].apply(lambda ts: int(ts.hour <= 11))

In [32]:
scaler = StandardScaler()
scaler.fit(new_train_features['start_hour'].values.reshape(-1, 1))

new_train_features['start_hour_scaled'] = scaler.transform(new_train_features['start_hour'].values.reshape(-1, 1))
new_test_features['start_hour_scaled'] = scaler.transform(new_test_features['start_hour'].values.reshape(-1, 1))

In [33]:
new_train_features.head()

,year_month,year_month_scaled,start_hour,morning,start_hour_scaled
session_id,,,,,
21669,201301,-1.744405,8,1,-1.357366
54843,201301,-1.744405,8,1,-1.357366
77292,201301,-1.744405,8,1,-1.357366
114021,201301,-1.744405,8,1,-1.357366
146670,201301,-1.744405,8,1,-1.357366


In [34]:
new_test_features.head()

,year_month,year_month_scaled,start_hour,morning,start_hour_scaled
session_id,,,,,
1,201410,0.822948,11,1,-0.407823
2,201407,0.752287,11,1,-0.407823
3,201412,0.870055,15,0,0.858234
4,201411,0.846501,10,1,-0.724338
5,201405,0.705179,15,0,0.858234


In [35]:
# Снова объединяем новые данные с существующими в разреженной матрице X_train_sparse_new
# Делаем это с помощью функции hstack из библиотеки scipy.sparse
# Требуется дополнительное явное преобразование в csr_matrix
X_train_sparse_new_2 = csr_matrix(hstack([X_train_sparse_new,
                                  new_train_features['start_hour_scaled'].values.reshape(-1, 1)]))

# Аналогично для тестового набора
X_test_sparse_new_2 = hstack([X_test_sparse_new,
                             new_test_features['start_hour_scaled'].values.reshape(-1, 1)])
X_train_sparse_new.shape, X_train_sparse_new_2.shape # Добавился дополнительный признак в shape[1]

((253561, 48372), (253561, 48373))

In [36]:
%%time
# Проверяем качество с 2-мя признаками по ROC AUC
get_auc_lr_valid(X=X_train_sparse_new_2, y=y_train)

CPU times: user 101 ms, sys: 120 ms, total: 221 ms
Wall time: 3.15 s


0.9573322845076919

In [37]:
# Добавим в разреженную матрицу ещё одну фичу morning с помощью того же hstack
X_train_sparse_new_3 = csr_matrix(hstack([X_train_sparse_new_2,
                                  new_train_features['morning'].values.reshape(-1, 1)]))

# Аналогично для тестового набора
X_test_sparse_new_3 = hstack([X_test_sparse_new_2,
                             new_test_features['morning'].values.reshape(-1, 1)])
X_train_sparse_new_2.shape, X_train_sparse_new_3.shape # Добавился дополнительный признак в shape[1]

((253561, 48373), (253561, 48374))

In [38]:
%%time
# Проверяем качество с 3-мя признаками по ROC AUC
get_auc_lr_valid(X=X_train_sparse_new_3, y=y_train)

CPU times: user 733 ms, sys: 162 ms, total: 895 ms
Wall time: 3.21 s


0.9585507820000507

In [39]:
# Обучим новую модель

logit = LogisticRegression(n_jobs=-1, C=1, random_state=17, solver='lbfgs')
logit.fit(X_train_sparse_new_3, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=-1, penalty='l2', random_state=17,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [40]:
# Создадим файл посылки для Kaggle с учётом 3-х новых фич
# Предсказанные результаты
pred_probabilities_3 = logit.predict_proba(X_test_sparse_new_3)
pred_probabilities_3[:, 1]

array([5.26264685e-05, 6.26559747e-08, 2.43439679e-09, ...,
       2.04955019e-04, 6.29702900e-06, 2.71673195e-07])

In [41]:
# write_to_submission_file(pred_probabilities_3[:, 1],
#                          out_file = "../../data/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/benchmark_1_5.csv",
#                          target='target',
#                          index_label="session_id")

### 4. Подбор коэффицициента регуляризации

Итак, мы ввели признаки, которые улучшают качество нашей модели по сравнению с первым бейслайном. Можем ли мы добиться большего значения метрики? После того, как мы сформировали обучающую и тестовую выборки, почти всегда имеет смысл подобрать оптимальные гиперпараметры -- характеристики модели, которые не изменяются во время обучения. Например, на 3 неделе вы проходили решающие деревья, глубина дерева это гиперпараметр, а признак, по которому происходит ветвление и его значение -- нет. В используемой нами логистической регрессии веса каждого признака изменяются и во время обучения находится их оптимальные значения, а коэффициент регуляризации остается постоянным. Это тот гиперпараметр, который мы сейчас будем оптимизировать.

Посчитайте качество на отложенной выборке с коэффициентом регуляризации, который по умолчанию `C=1`:

In [42]:
from sklearn.metrics import accuracy_score

In [43]:
%%time
# наша целевая переменная на отложенной выборке (для тестирования)
get_accuracy_value(X=X_train_sparse_new_3, y=y_train)

CPU times: user 96 ms, sys: 110 ms, total: 206 ms
Wall time: 2.91 s


0.9908372661662438

In [44]:
%%time
# Проверяем качество с 3-мя признаками по ROC AUC - так проверяется на Kaggle
get_auc_lr_valid(X=X_train_sparse_new_3, y=y_train, C=1, ratio=0.9)

CPU times: user 113 ms, sys: 124 ms, total: 238 ms
Wall time: 3.36 s


0.9585507820000507

Постараемся побить этот результат за счет оптимизации коэффициента регуляризации. Возьмем набор возможных значений C и для каждого из них посчитаем значение метрики на отложенной выборке.

Найдите `C` из `np.logspace(-3, 1, 10)`, при котором ROC AUC на отложенной выборке максимален. 

In [45]:
reg_coefficient = np.logspace(-3, 1, 10)

result = (0, 0)
for c in reg_coefficient:
    local_result = get_auc_lr_valid(X=X_train_sparse_new_3, ratio=0.9, y=y_train, C=c)
    if result[0] < local_result:
        result = (local_result, c)

result_reg_coefficient = result[1]
result

(0.960868674591127, 0.1668100537200059)

Наконец, обучите модель с найденным оптимальным значением коэффициента регуляризации и с построенными признаками `start_hour`, `start_month` и `morning`. Если вы все сделали правильно и загрузите это решение, то повторите второй бенчмарк соревнования.

In [46]:
# Создадим файл посылки для Kaggle с учётом 3-х новых фич и коэффициента регуляризации
# Предсказанные результаты
logit = LogisticRegression(n_jobs=-1, C=result_reg_coefficient, random_state=17, solver='lbfgs')
logit.fit(X_train_sparse_new_3, y_train)

pred_probabilities_2_nd_benchmark = logit.predict_proba(X_test_sparse_new_3)
pred_probabilities_2_nd_benchmark[:, 1]

array([2.54857950e-04, 2.35772047e-06, 9.26844124e-07, ...,
       5.60875922e-04, 6.04939606e-05, 7.53602794e-06])

In [48]:
# write_to_submission_file(pred_probabilities_2_nd_benchmark[:, 1],
#                          out_file = "../../data/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/benchmark_2.csv",
#                          target='target',
#                          index_label="session_id")

### 5. А теперь попробуем подобрать ещё несколько фич

In [49]:
# Попробуем добавить фичу с количеством сайтов, просматриваемых за одну сессию sites_amount_per_session

amount_of_unique_sites_train = X_train_sparse.sum(axis=1)
amount_of_unique_sites_test = X_test_sparse.sum(axis=1)

new_train_features['sites_amount_per_session'] = amount_of_unique_sites_train
new_test_features['sites_amount_per_session'] = amount_of_unique_sites_test

new_train_features['short_session'] = amount_of_unique_sites_train < 10
new_test_features['short_session'] = amount_of_unique_sites_test < 10

new_train_features['short_session'] = new_train_features['short_session'].astype('int')
new_test_features['short_session'] = new_test_features['short_session'].astype('int')
new_train_features.head()

,year_month,year_month_scaled,start_hour,morning,start_hour_scaled,sites_amount_per_session,short_session
session_id,,,,,,,
21669,201301,-1.744405,8,1,-1.357366,2,1
54843,201301,-1.744405,8,1,-1.357366,4,1
77292,201301,-1.744405,8,1,-1.357366,10,0
114021,201301,-1.744405,8,1,-1.357366,10,0
146670,201301,-1.744405,8,1,-1.357366,10,0


In [50]:
scaler_5 = StandardScaler()
scaler_5.fit(new_train_features['sites_amount_per_session'].values.reshape(-1, 1))

new_train_features['sites_amount_per_session_scaled'] = scaler_5.transform(new_train_features['sites_amount_per_session'].values.reshape(-1, 1))
new_test_features['sites_amount_per_session_scaled'] = scaler_5.transform(new_test_features['sites_amount_per_session'].values.reshape(-1, 1)) 
new_train_features.head()

,year_month,year_month_scaled,start_hour,morning,start_hour_scaled,sites_amount_per_session,short_session,sites_amount_per_session_scaled
session_id,,,,,,,,
21669,201301,-1.744405,8,1,-1.357366,2,1,-4.328969
54843,201301,-1.744405,8,1,-1.357366,4,1,-3.177031
77292,201301,-1.744405,8,1,-1.357366,10,0,0.278784
114021,201301,-1.744405,8,1,-1.357366,10,0,0.278784
146670,201301,-1.744405,8,1,-1.357366,10,0,0.278784


In [51]:
# Добавим в разреженную матрицу ещё одну фичу morning с помощью того же hstack
X_train_sparse_new_4 = csr_matrix(hstack([X_train_sparse_new_3,
                                  new_train_features[['sites_amount_per_session_scaled', 'short_session']].values]))

# Аналогично для тестового набора
X_test_sparse_new_4 = hstack([X_test_sparse_new_3,
                             new_test_features[['sites_amount_per_session_scaled', 'short_session']].values])
X_train_sparse_new_3.shape, X_train_sparse_new_4.shape # Добавились дополнительные признаки в shape[1]

((253561, 48374), (253561, 48376))

In [52]:
%%time
# Проверяем качество с 3-мя признаками по ROC AUC - так проверяется на Kaggle
get_auc_lr_valid(X=X_train_sparse_new_4, y=y_train, C=result_reg_coefficient, ratio=0.9)

CPU times: user 450 ms, sys: 140 ms, total: 590 ms
Wall time: 3.69 s


0.9617147695421517

In [53]:
result_1 = (0, 0)
for c in reg_coefficient:
    local_result = get_auc_lr_valid(X=X_train_sparse_new_4, ratio=0.9, y=y_train, C=c)
    if result_1[0] < local_result:
        result_1 = (local_result, c)

result_reg_coefficient_1 = result_1[1]
result_1

(0.9620494336824589, 0.05994842503189409)

In [54]:
%%time
# Проверяем качество с 3-мя признаками по ROC AUC - так проверяется на Kaggle
get_auc_lr_valid(X=X_train_sparse_new_4, y=y_train, C=result_reg_coefficient_1, ratio=0.9)

CPU times: user 112 ms, sys: 123 ms, total: 234 ms
Wall time: 3 s


0.9620494336824589

In [55]:
logit = LogisticRegression(n_jobs=-1, C=result_reg_coefficient_1, random_state=17, solver='lbfgs')
logit.fit(X_train_sparse_new_4, y_train)

pred_probabilities_benchmark_2_1 = logit.predict_proba(X_test_sparse_new_4)
pred_probabilities_benchmark_2_1[:, 1]

# write_to_submission_file(pred_probabilities_benchmark_2_1[:, 1],
#                          out_file = "../../data/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/benchmark_2_1.csv",
#                          target='target',
#                          index_label="session_id")

array([5.06128982e-04, 1.03396300e-05, 1.31627787e-05, ...,
       9.07472535e-04, 1.53460865e-04, 3.15474427e-05])

In [56]:
new_train_zero_na_df = train_na_zero.copy(deep=True)
new_test_zero_na_df = train_na_zero.copy(deep=True)

In [57]:
for i in range(9, 0, -1):
    intermediate_train_df = new_train_zero_na_df[(new_train_zero_na_df['time10'] == pd.Timestamp(0)) & \
                                                 (new_train_zero_na_df['time%s' % i] != pd.Timestamp(0))]
    new_train_zero_na_df.loc[intermediate_train_df.index.values,
                             ['time10']] = intermediate_train_df['time%s' % i]
    
    intermediate_test_df = new_test_zero_na_df[(new_test_zero_na_df['time10'] == pd.Timestamp(0)) & \
                                               (new_test_zero_na_df['time%s' % i] != pd.Timestamp(0))]
    new_test_zero_na_df.loc[intermediate_test_df.index.values,
                            ['time10']] = intermediate_test_df['time%s' % i]
    
new_train_zero_na_df.head()

,time1,time2,time3,time4,time5,time6,time7,time8,time9,time10
session_id,,,,,,,,,,
21669,2013-01-12 08:05:57,2013-01-12 08:05:57,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,2013-01-12 08:05:57
54843,2013-01-12 08:37:23,2013-01-12 08:37:23,2013-01-12 09:07:07,2013-01-12 09:07:09,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,1970-01-01 00:00:00,2013-01-12 09:07:09
77292,2013-01-12 08:50:13,2013-01-12 08:50:14,2013-01-12 08:50:15,2013-01-12 08:50:15,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:17,2013-01-12 08:50:17
114021,2013-01-12 08:50:17,2013-01-12 08:50:17,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:19,2013-01-12 08:50:19,2013-01-12 08:50:19,2013-01-12 08:50:20
146670,2013-01-12 08:50:20,2013-01-12 08:50:20,2013-01-12 08:50:20,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:22,2013-01-12 08:50:22,2013-01-12 08:50:22


In [58]:
# Добавляем новую фичу - is_quick_launch, которая показываеет, были ли открыти все сайты в сессии в течение 5 минут
# или открывались один за другим в течение более долгого времени
new_train_features['sites_open_time_frame'] = (new_train_zero_na_df['time10'] - \
                                               new_train_zero_na_df['time1']).apply(lambda dt: dt.total_seconds() / 60.0)

new_test_features['sites_open_time_frame'] = (new_test_zero_na_df['time10'] - \
                                              new_test_zero_na_df['time1']).apply(lambda dt: dt.total_seconds() / 60.0)

new_train_features['is_quick_launch'] = (new_train_features['sites_open_time_frame'] < 2).astype(np.int)

new_test_features['is_quick_launch'] = (new_test_features['sites_open_time_frame'] < 2).astype(np.int)

new_train_features.head()

,year_month,year_month_scaled,start_hour,morning,start_hour_scaled,sites_amount_per_session,short_session,sites_amount_per_session_scaled,sites_open_time_frame,is_quick_launch
session_id,,,,,,,,,,
21669,201301,-1.744405,8,1,-1.357366,2,1,-4.328969,0.000000,1
54843,201301,-1.744405,8,1,-1.357366,4,1,-3.177031,29.766667,0
77292,201301,-1.744405,8,1,-1.357366,10,0,0.278784,0.066667,1
114021,201301,-1.744405,8,1,-1.357366,10,0,0.278784,0.050000,1
146670,201301,-1.744405,8,1,-1.357366,10,0,0.278784,0.033333,1


In [90]:
scaler_6 = StandardScaler()
scaler_6.fit(new_train_features['sites_open_time_frame'].values.reshape(-1, 1))

new_train_features['sites_open_time_frame_scaled'] = scaler_6.transform(new_train_features['sites_open_time_frame'].values.reshape(-1, 1))
new_test_features['sites_open_time_frame_scaled'] = scaler_6.transform(new_test_features['sites_open_time_frame'].values.reshape(-1, 1)) 
new_train_features.head()

,year_month,year_month_scaled,start_hour,morning,start_hour_scaled,sites_amount_per_session,short_session,sites_amount_per_session_scaled,sites_open_time_frame,is_quick_launch,sites_open_time_frame_scaled
session_id,,,,,,,,,,,
21669,201301,-1.744405,8,1,-1.357366,2,1,-4.328969,0.000000,1,-0.468233
54843,201301,-1.744405,8,1,-1.357366,4,1,-3.177031,29.766667,0,5.570015
77292,201301,-1.744405,8,1,-1.357366,10,0,0.278784,0.066667,1,-0.454709
114021,201301,-1.744405,8,1,-1.357366,10,0,0.278784,0.050000,1,-0.458090
146670,201301,-1.744405,8,1,-1.357366,10,0,0.278784,0.033333,1,-0.461471


In [60]:
# Добавим в разреженную матрицу ещё одну фичу is_quick_launch с помощью того же hstack
X_train_sparse_new_5 = csr_matrix(hstack([X_train_sparse_new_4,
                                  new_train_features[['is_quick_launch']].values]))

# Аналогично для тестового набора
X_test_sparse_new_5 = hstack([X_test_sparse_new_4,
                             new_test_features[['is_quick_launch']].values])
X_train_sparse_new_4.shape, X_train_sparse_new_5.shape # Добавились дополнительные признаки в shape[1]

((253561, 48376), (253561, 48377))

In [61]:
result_2 = (0, 0)
for c in reg_coefficient:
    local_result = get_auc_lr_valid(X=X_train_sparse_new_5, ratio=0.9, y=y_train, C=c)
    if result_2[0] < local_result:
        result_2 = (local_result, c)

result_reg_coefficient_2 = result_2[1]
result_2

(0.9623047717038073, 0.05994842503189409)

In [62]:
%%time
# Проверяем качество с 8-мью признаками по ROC AUC - так проверяется на Kaggle
get_auc_lr_valid(X=X_train_sparse_new_5, y=y_train, C=result_reg_coefficient_2, ratio=0.9)

CPU times: user 119 ms, sys: 131 ms, total: 250 ms
Wall time: 3.05 s


0.9623047717038073

In [63]:
logit = LogisticRegression(n_jobs=-1, C=result_reg_coefficient_2, random_state=17, solver='lbfgs')
logit.fit(X_train_sparse_new_5, y_train)

pred_probabilities_benchmark_2_1_1 = logit.predict_proba(X_test_sparse_new_5)
pred_probabilities_benchmark_2_1_1[:, 1]

# write_to_submission_file(pred_probabilities_benchmark_2_1_1[:, 1],
#                          out_file = "../../data/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/benchmark_2_1_1.csv",
#                          target='target',
#                          index_label="session_id")

### 6. Попробуем применить подход TF-IDF

In [64]:
import scipy as sci

In [65]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


In [66]:
# Разреженная матрица с мерами Term Frequency
tf_csr_matrix = np.dot(full_sites_sparse, 1 / full_sites_sparse.shape[1])
# print(tf_csr_matrix)

# Вектор количества упомининий каждого сайта во всех сессиях
overall_freq_vector = np.sum(full_sites_sparse > 0, axis=0)
overall_freq_vector = np.array(overall_freq_vector)
overall_freq_vector

array([[ 3725,   389, 15596, ...,     1,     5,     1]], dtype=int64)

In [71]:
# Создадим разреженную матрицу inverse document frequency:
inverse_document_frequency_vector = np.log10(np.divide(full_sites_sparse.shape[0], overall_freq_vector))
inverse_document_frequency_vector[inverse_document_frequency_vector == np.inf] = 0
inverse_document_frequency_vector_csr = sci.sparse.csr_matrix(inverse_document_frequency_vector)
inverse_document_frequency_vector_csr

<1x48371 sparse matrix of type '<class 'numpy.float64'>'
	with 48371 stored elements in Compressed Sparse Row format>

In [78]:
# inverse_document_frequency_matrix = sci.sparse.csr_matrix(np.diag(list(inverse_document_frequency_vector[0])))
# inverse_document_frequency_matrix

In [84]:
# Матрица TF-IDF
tf_idf_matrix_sparse = tf_csr_matrix.multiply(inverse_document_frequency_vector_csr)
tf_idf_matrix_sparse

<336358x48371 sparse matrix of type '<class 'numpy.float64'>'
	with 1866898 stored elements in Compressed Sparse Row format>

In [85]:
X_train_sparse_tf_idf = tf_idf_matrix_sparse[:idx_split]
X_test_sparse_tf_idf = tf_idf_matrix_sparse[idx_split:]
X_train_sparse_tf_idf.shape, y_train.shape

((253561, 48371), (253561,))

In [107]:
%%time
get_auc_lr_valid(X=X_train_sparse_tf_idf, ratio=0.9, y=y_train, C=0.001)

CPU times: user 98 ms, sys: 169 ms, total: 267 ms
Wall time: 1.85 s


0.892248940003792

In [110]:
scaler = StandardScaler(with_mean=False)
scaler.fit(X_train_sparse_tf_idf)

X_train_sparse_tf_idf_1 = scaler.transform(X_train_sparse_tf_idf)
X_test_sparse_tf_idf_1 = scaler.transform(X_test_sparse_tf_idf)

In [111]:
%%time
get_auc_lr_valid(X=X_train_sparse_tf_idf_1, ratio=0.9, y=y_train, C=0.001)

CPU times: user 95.6 ms, sys: 153 ms, total: 249 ms
Wall time: 2.53 s


0.8977614637186031

In [113]:
# Добавим в разреженную матрицу ещё одну фичу is_quick_launch с помощью того же hstack
X_train_sparse_tf_idf_2 = csr_matrix(hstack([X_train_sparse_tf_idf_1,
                                     new_train_features[['is_quick_launch',
                                                         'sites_amount_per_session_scaled',
                                                         'short_session',
                                                         'start_hour_scaled',
                                                         'morning',
                                                         'year_month_scaled']].values]))

# Аналогично для тестового набора
X_test_sparse_tf_idf_2 = csr_matrix(hstack([X_test_sparse_tf_idf_1,
                                    new_test_features[['is_quick_launch',
                                                       'sites_amount_per_session_scaled',
                                                       'short_session',
                                                       'start_hour_scaled',
                                                       'morning',
                                                       'year_month_scaled']].values]))
X_train_sparse_tf_idf_1.shape, X_train_sparse_tf_idf_2.shape # Добавились дополнительные признаки в shape[1]

((253561, 48371), (253561, 48377))

In [114]:
result_3 = (0, 0)
for c in reg_coefficient:
    local_result = get_auc_lr_valid(X=X_train_sparse_tf_idf_2, ratio=0.9, y=y_train, C=c)
    if result_3[0] < local_result:
        result_3 = (local_result, c)

result_reg_coefficient_3 = result_3[1]
result_3

(0.9389415314424281, 0.0027825594022071257)

In [117]:
logit = LogisticRegression(n_jobs=-1, C=result_reg_coefficient_3, random_state=17, solver='lbfgs')
logit.fit(X_train_sparse_tf_idf_2, y_train)

pred_probabilities_benchmark_2_1_1_tf_idf = logit.predict_proba(X_test_sparse_tf_idf_2)
pred_probabilities_benchmark_2_1_1_tf_idf[:, 1].shape

(82797,)

In [ ]:
write_to_submission_file(pred_probabilities_benchmark_2_1_1_tf_idf[:, 1],
                         out_file = "../../data/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/benchmark_2_1_1_tf_idf.csv",
                         target='target',
                         index_label="session_id")